In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.2 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=e61a5d8b2daa3c5c7e41fc463c0a856f3694dc7e2cc8b5e042b4a2ed0db8aec9
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.3 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_US         dataframe_result_csv_UK
 a0529.csv	        Github
 a0530.csv	        heatmap529_UK.ipynb
 a0531.csv	        heatmap530.ipynb
 a0601.csv	        Heat_Map_Online_language.ipynb
 a0602.csv	        __MACOSX
 csv		        non_violant1.csv
 csv.zip	        non_violant2.csv
 dataframe_csv	        UK
 dataframe_csv_UK       violant.csv
 dataframe_result_csv  '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_03.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,419544961466200_2985172334903437,Facebook,2020-06-03 23:59:54,2021-08-24 06:13:28,link,George Floyd Updates: 3 More Officers Charged;...,nytimes.com,As unrest over race and policing roils the nat...,Finally! What say you? https://www.nytimes.com...,2.0,https://www.nytimes.com/2020/06/03/us/live-geo...,https://www.facebook.com/419544961466200/posts...,183197,1.720588,1.0,78,13,5,16,5,0,0,0,0,2,28,5,14,6,2,5,3,5,0,2,403956,Victor Willis,officialvictorwillis,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Official Facebook Page of VICTOR WILLIS, lead ...",True,2013-02-17 05:01:34,en,0,403956|2985172334903437
1,719648111502249_1941990192601362,Facebook,2020-06-03 23:59:53,2021-07-10 00:16:35,link,Weber State Athletics issues statements denoun...,standard.net,OGDEN — As Americans grapple with the issues o...,George Floyd died at the knee of a police offi...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/719648111502249/posts...,1729,-30.000000,1.0,0,0,0,0,0,0,0,0,0,0,5,2,4,2,1,0,1,0,0,1,4685612,Standard-Examiner Sports,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,4685612|1941990192601362
2,229748831408114_278454989870831,Facebook,2020-06-03 23:59:53,2021-04-03 03:49:13,status,Weber State Athletics issues statements denoun...,standard.net,OGDEN — As Americans grapple with the issues o...,It is so hard to figure out how to show up rig...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/groups/22974883140811...,4784,-1.341463,1.0,26,0,4,11,0,0,0,0,0,10,17,4,6,10,5,7,5,1,0,6,10735585,COVID-19 Acts of Grace and Generosity,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,10735585|278454989870831
3,1250734815048111_3089596414495266,Facebook,2020-06-03 23:59:50,2021-08-16 19:32:51,link,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",**This great man was a lifelong best friend of...,2.0,https://www.espn.com/college-football/story/_/...,https://www.facebook.com/groups/12507348150481...,28550,69.038462,1.0,62,673,133,3,11,0,672,241,0,7,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,7838630|3089596414495266
4,358599564868959_632768947452018,Facebook,2020-06-03 23:59:49,2021-07-04 09:11:04,photo,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",Handcuffed. Face Down. Knee on his neck. They ...,1.0,https://www.facebook.com/photo.php?fbid=102235...,https://www.facebook.com/groups/35859956486895...,4798,1.340659,1.0,91,96,22,2,1,0,96,58,0,2,136,35,22,37,4,19,6,14,0,4,7504729,Build Back Better Florida,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,7504729|632768947452018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{'Virginia': 1}
1,{},{'Minnesota': 1}
2,{},{'Minnesota': 1}
3,{'Indianapolis': 1},{'Indianapolis': 1}
4,{'America': 1},{'Indianapolis': 1}


In [24]:
ct.to_csv('a0603.csv')

In [25]:
ct=pd.read_csv('a0603.csv')

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{'Virginia': 1},{}
1,{},{'Minnesota': 1},{}
2,{},{'Minnesota': 1},{}
3,{'Indianapolis': 1},{'Indianapolis': 1},{'Indianapolis': 1}
4,{'America': 1},{'Indianapolis': 1},{'Indianapolis': 1}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0                                              []
1                                              []
2                                              []
3                                  [Indianapolis]
4                                       [America]
                           ...                   
50763    [Minneapolis, Minnesota, South Carolina]
50764                                          []
50765                                          []
50766                                          []
50767                                          []
Name: NER-msg, Length: 35133, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0                                               [Virginia]
1                                              [Minnesota]
2                                              [Minnesota]
3               [Indianapolis, Indianapolis, Indianapolis]
4                    [America, Indianapolis, Indianapolis]
                               ...                        
50763    [Minneapolis, Minnesota, South Carolina, South...
50764                                     [South Carolina]
50765                                     [South Carolina]
50766                             [Minneapolis, Minnesota]
50767                                         [Morristown]
Name: NER, Length: 35133, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['virginia']), list(['minnesota']), list(['minnesota']), ...,
       list(['south carolina']), list(['minneapolis', 'minnesota']),
       list(['morristown'])], dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['virginia',
 'minnesota',
 'minnesota',
 'indianapolis',
 'america',
 'indianapolis',
 'indianapolis',
 'minnesota',
 'indianapolis',
 'florida',
 'minneapolis',
 'los angeles',
 'indianapolis',
 "san francisco's",
 'san francisco',
 'oakland',
 'us',
 'minnesota',
 'minneapolis',
 'texas',
 'el paso',
 'minneapolis',
 'texas',
 'el paso',
 'minneapolis',
 'texas',
 'el paso',
 'mississippi',
 'mississippi',
 'minneapolis',
 'grenada',
 'minneapolis',
 'denver',
 'denver',
 'u.s',
 'u.s.',
 'minneapolis',
 'city',
 '| city',
 'montgomery',
 'ohio',
 'the city of montgomery',
 'san diego',
 'america',
 'west africa',
 'us',
 'santa clara law',
 'chicago',
 'austin',
 'new york',
 'washington d.c.',
 'minneapolis',
 'atlanta',
 'cleveland',
 'the united states',
 'fayetteville',
 'america',
 'durham',
 'philadelphia',
 'los angeles',
 'raleigh',
 'st. louis',
 'america',
 'lafayette park',
 'd.c.',
 'washington',
 'america',
 'america',
 'america',
 'america',
 'america',
 'new orleans'

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 13305),
 ('minnesota', 4894),
 ('us', 4272),
 ('america', 4127),
 ('the united states', 2610),
 ('u.s.', 2508),
 ('washington', 1242),
 ('houston', 1058),
 ('new york', 899),
 ('uk', 795),
 ('london', 707),
 ('los angeles', 703),
 ('new york city', 688),
 ('usa', 609),
 ('chicago', 599),
 ('atlanta', 465),
 ('texas', 458),
 ('d.c.', 444),
 ('paris', 418),
 ('california', 363),
 ('nj', 363),
 ('florida', 322),
 ('philadelphia', 321),
 ('canada', 302),
 ('portland', 301),
 ('nyc', 284),
 ('georgia', 281),
 ('louisville', 275),
 ('brooklyn', 254),
 ('the united states of america', 254),
 ('nigeria', 251),
 ('virginia', 244),
 ('australia', 244),
 ('manhattan', 240),
 ('united states', 239),
 ('maryland', 236),
 ('denver', 235),
 ('new jersey', 231),
 ('st. louis', 226),
 ('boston', 221),
 ('north carolina', 216),
 ('alabama', 212),
 ('dallas', 205),
 ('dc', 201),
 ('china', 200),
 ('las vegas', 196),
 ('seattle', 195),
 ('oakland', 187),
 ('hollywood', 185),
 ('germany', 

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

5088

In [44]:
dataframe531.to_csv('dataframe0603_UK.csv')